In [60]:
import torch
import numpy as np

In [95]:
def Ising(nRow, nCol):
    ising = torch.zeros(nRow, nCol)
    x = torch.rand(nRow, nCol)
    for i in range(nRow):
        for j in range(nCol):
            if x[i,j] < 0.5:
                    ising[i,j] = 0
            else:
                    ising[i,j] = 1
    return ising

In [97]:
def Hamiltonian(ising):
    H = 0
    J = 1
    for i in range(nRow):
        for j in range(nCol):
                if i < 1:
                    H -= J * ising[i,j] * ising[i+1,j]
                elif i > nRow - 2:
                    H -= J * ising[i,j] * ising[i-1,j]
                else:
                    H -= J * ising[i,j] * ising[i+1,j]
                    H -= J * ising[i,j] * ising[i-1,j]
                
                if j < 1:
                    H -= J * ising[i,j] * ising[i,j+1]
                elif j > nCol - 2:
                    H -= J * ising[i,j] * ising[i,j-1]
                else:
                    H -= J * ising[i,j] * ising[i,j+1]
                    H -= J * ising[i,j] * ising[i,j-1]
    return H/2   #to avoid double count

In [55]:
visible = ising.view(nRow*nCol)
print(visible)

tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=torch.int32)


In [56]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [57]:
nv = len(visible)
nh = 10
batch_size = 10
rbm = RBM(nv, nh)

In [59]:
rbm.b

tensor([[ 1.5756,  0.6087, -0.6850, -2.3785,  0.6885, -0.6935, -0.0844, -0.8821,
          2.8655,  0.7690, -1.2783, -0.9531,  1.5025,  0.8976,  0.5784,  0.0060]])